<a href="https://colab.research.google.com/github/matteoalberti/Lectures_AdvCV_Experis2020/blob/main/day1/DL4CV_1_Introduction_to_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://github.com/matteoalberti/Lectures_AdvCV_Experis2020/blob/main/day1/images/intro.jpg?raw=1)

# **Welcome!**

## Introduction to Machine Learning for Computer Vision : Framework



## **Lecturer :** Matteo Alberti

# Summary

- <font color=BE3315>**Load Data** </font> 

- <font color=C24024>**Define HyperParameters** </font> 

- <font color=E35F2A>**KERAS Models : Sequential vs Functional** </font> 

- <font color=EF8932>**MLP | Train and Test** </font>


#### Load Packages

**Allocare resources! GPUs**

In [ ]:
import os
import pickle
import pandas as pd
import argparse
from random import randint
import time
import gc
import cv2
import platform
import numpy as np

#Visualization
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import seaborn as sns
sns.set_style("white")

#Sklearn ML
from sklearn.model_selection import train_test_split

#Keras Imports
import tensorflow as tf

""" NO MORE!
import keras
from keras.initializers import he_normal
from keras import optimizers
from keras.utils import np_utils
from keras.callbacks import LearningRateScheduler
from keras.models import load_model
from keras.layers import Input,add, AveragePooling2D, GlobalAveragePooling2D, Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Activation, Conv2DTranspose
from keras.models import Model, Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.optimizers import Adam , SGD
from keras import optimizers, regularizers
from keras.utils import to_categorical
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
"""

print('TF Version : ',tf.__version__)
print('Python Version : ', platform.python_version())

Keras Version :  2.4.3
TF Version :  2.3.0
Python Version :  3.6.9


### Load Cifar10 dataset


In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

170500096/170498071 [==============================] - 6s 0us/step
(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)


##### Split out Data

*We don't need it, why? let's see . . .*

##### Normalize data

In [ ]:
#The range for each individual colour is 0-255
x_train = x_train.astype('float32')/255 
x_test = x_test.astype('float32')/255

### Define HyperParameters & CNN Architecture
   

In [ ]:
#Parameters
import sys;import argparse; sys.argv=['']; del sys
parser = argparse.ArgumentParser(description="CNN")
parser.add_argument('--epochs', default=20, type=int)
parser.add_argument('--iter', default=100, type=int)
parser.add_argument('--batch_size', default=128, type=int)
parser.add_argument('--lr', default=0.001, type=float)

parser.add_argument('--height', default=32, type=int)
parser.add_argument('--width', default=32, type=int)
parser.add_argument('--channel', default=3, type=int)
parser.add_argument('--classes', default=10, type=int)

args = parser.parse_args()

print(args)

Namespace(batch_size=128, channel=3, classes=10, epochs=20, height=32, iter=100, lr=0.001, width=32)


## Keras Models: Sequential vs. Functional API

- The **sequential** API allows you to create models layer-by-layer
- The **functional** API allows you to create models that have a lot more flexibility as you can easily define models where layers connect to more than just the previous and next layers

In [ ]:
x_train = x_train.reshape(x_train.shape[0], 32*32*3)

x_test = x_test.reshape(x_test.shape[0], 
                        x_test.shape[1]*x_test.shape[2]*x_test.shape[3])


print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(50000, 3072) (5000, 3072) (50000, 10) (5000, 10)


### KERAS SEQUENTIAL API

In [ ]:
# SEQUENTIAL

model = Sequential()
model.add(Dense(512, activation='relu', 
                input_shape=(args.height*args.width*args.channel,)))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.001), 
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               1573376   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 1,578,506
Trainable params: 1,578,506
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# or eventually

def mlp():
    return tf.keras.Sequential([
        
        ####
        ####
        ####
    ])

In [ ]:
start = time.time()
mlp_baseline = model.fit(x_train, y_train, batch_size=64,epochs=10,verbose=2, validation_split=0.2)
end = time.time()
print ("Model took %0.2f seconds to train"%(end - start))

In [ ]:
mlp_baseline.history

In [ ]:
def plot_model_history(model_history):
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    # summarize history for accuracy
    axs[0].plot(range(1,len(model_history.history['accuracy'])+1),model_history.history['accuracy'])
    axs[0].plot(range(1,len(model_history.history['val_accuracy'])+1),model_history.history['val_accuracy'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_history.history['accuracy'])+1),len(model_history.history['accuracy'])/10)
    axs[0].legend(['train', 'val'], loc='best')
    # summarize history for loss
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    plt.show()

In [ ]:
plot_model_history(mlp_baseline)

### How appears our training history?

![](https://github.com/matteoalberti/Lectures_AdvCV_Experis2020/blob/main/day1/images/underfit.png?raw=1)

In [ ]:
def save_history(history, result_file):
    loss = history.history['loss']
    acc = history.history['accuracy']
    val_loss = history.history['val_loss']
    val_acc = history.history['val_accuracy']
    nb_epoch = len(acc)

    with open(result_file, "w") as fp:
        fp.write("epoch\tloss\tacc\tval_loss\tval_acc\n")
        for i in range(nb_epoch):
            fp.write("%d\t%f\t%f\t%f\t%f\n" %
                     (i, loss[i], acc[i], val_loss[i], val_acc[i]))

save_history(mlp_baseline, 'history.txt')

In [ ]:
loss, acc = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', loss)
print('Test acc:', acc)

In [ ]:
! mkdir pretrained_model

In [ ]:
# save model locally!
mlp_baseline.save_weights('./pretrained_model/mlp_baseline.h5')

#### Exists any better way? Less custom?

- Yes... tensorboard!

### KERAS FUNCTIONAL API

In [ ]:
_inputs = Input(shape=(args.height*args.width*args.channel,))
hidden = Dense(512, activation='relu')(_inputs)
_output = Dense(10, activation='softmax')(hidden)
model = Model(inputs=_inputs, outputs=_output)

model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3072)]            0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               1573376   
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 1,578,506
Trainable params: 1,578,506
Non-trainable params: 0
_________________________________________________________________


### Which are the next steps?

*Are you able to take the above code and re-train our mlp?*